In [1]:
import pandas as pd

In [2]:
string_set_pre_reference = [('california academy of science[s]?', 200), ('southern california academy of science[s]?', -200), ('CASC', 60), ('CASIZ', 200), ('izcas', -200), ('CAS-SUA', 200), ('CAS-SUR', 200), ('CAS-SU', 200), ('CAS:SU', 200), ('CASG', 200), ('CAS:ICH', 200), ('CAS-ICH', 200), ('CASTYPE', 200), ('CASENT', 200)]
collection_manager_names = {('d. catania', 200), ('s. cotterell', 300), ('l scheinberg', 1000), ('m abarca', 200), ('jon fong', 200), ('rebekah kim', 200), ('jens vindum', 1000), ('c garcia', 200), ('j loacker', 1000), ('c piotrowski', 1000), ('d catania', 200), ('jim shevock', 300), ('s cotterell', 300), ('wojciech pulawski', 100)}

In [3]:
string_set_pre_reference = set(string_set_pre_reference)
string_set_pre_reference.update(collection_manager_names)

In [4]:
string_set_pre_reference

{('CAS-ICH', 200),
 ('CAS-SU', 200),
 ('CAS-SUA', 200),
 ('CAS-SUR', 200),
 ('CAS:ICH', 200),
 ('CAS:SU', 200),
 ('CASC', 60),
 ('CASENT', 200),
 ('CASG', 200),
 ('CASIZ', 200),
 ('CASTYPE', 200),
 ('c garcia', 200),
 ('c piotrowski', 1000),
 ('california academy of science[s]?', 200),
 ('d catania', 200),
 ('d. catania', 200),
 ('izcas', -200),
 ('j loacker', 1000),
 ('jens vindum', 1000),
 ('jim shevock', 300),
 ('jon fong', 200),
 ('l scheinberg', 1000),
 ('m abarca', 200),
 ('rebekah kim', 200),
 ('s cotterell', 300),
 ('s. cotterell', 300),
 ('southern california academy of science[s]?', -200),
 ('wojciech pulawski', 100)}

In [2]:
import logging
import mysql.connector
import yaml
import traceback
import time
from pymysql.converters import escape_string

class DBConnector(object):

    def __init__(self):
        self.dbconn = None
        self.db_config = self.read_db_config()

    def read_db_config(self):
        with open('./vm/vm_passwords.yml', 'r') as file:
            config = yaml.safe_load(file)
        return config

    # def create_connection(self):
    #     return sqlite3.connect('doi_database.db', timeout=30.0)

    def create_connection(self):
        return mysql.connector.connect(
            pool_name='mypool',
            pool_size=10,
            host=self.db_config['database_url'],
            user=self.db_config['database_user'],  # Replace with your username
            password=self.db_config['database_password'],

            database=self.db_config['database_name'],  # Replace with your database name
            port=self.db_config['database_port']
        )

    # For explicitly opening database connection
    def __enter__(self):
        self.dbconn = self.create_connection()
        return self.dbconn

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.dbconn.close()


db_conn = DBConnector()


class DBConnection(object):
    connection = None

    @classmethod
    def get_connection(cls, new=False):
        """Creates return new Singleton database connection"""
        if new or not cls.connection:
            cls.connection = db_conn.create_connection()
        return cls.connection

    @classmethod
    def log_sql(cls, query, stack_trace):
        # for debugging, enable if there are issues.
        # works with ./vm/sql_log_scan.sh
        return
        formatted_stack_trace = ''.join(stack_trace)
        log_string = (
            f"SQL Query: {query}\n"
            "Stack Trace:\n"
            f"{formatted_stack_trace}\n"
            f"{'-' * 40}\n"
        )
        with open('./sql.log', 'a') as log_file:
            log_file.write(log_string)
    @classmethod
    def execute_query(cls, query, args=None):
        """
        Execute a SQL query with retry mechanism on deadlock.

        :param query: The SQL query
        :param args: Arguments for the query
        :return: Query result for SELECT, or None for other types
        """
        max_retries = 3
        retry_delay = 60  # seconds

        for attempt in range(max_retries):
            connection = cls.get_connection()
            try:
                cursor = connection.cursor()
                if args is None:
                    cursor.execute(query)
                    formatted_query = query
                else:
                    cursor.execute(query, args)
                    formatted_query = query
                    for arg in args:
                        formatted_query = formatted_query.replace('%s', f"'{escape_string(str(arg))}'", 1)

                cls.log_sql(formatted_query, traceback.format_stack())
                if query.strip().upper().startswith("SELECT"):
                    result = cursor.fetchall()
                    cursor.close()
                    return result
                else:
                    connection.commit()
                    cursor.close()
                    return None
            except mysql.connector.errors.InternalError as e:
                if e.errno == 1213:  # Deadlock error code
                    logging.warning(
                        f"Deadlock detected, attempt {attempt + 1} of {max_retries}. Retrying in {retry_delay} seconds.")
                    time.sleep(retry_delay)
                else:
                    raise e
            except Exception as e:
                logging.critical(f"Bad SQL: {e}:\n{query}")
                print(traceback.format_exc())

                cursor.close()
                raise e

        # If all retries fail, rethrow the last exception
        raise Exception(f"Failed to execute query after {max_retries} attempts due to deadlock.")




In [3]:
conn = DBConnection()

In [4]:
curr = conn.get_connection()

In [5]:
curr

In [6]:
from config import Config
config = Config()

In [7]:
from doi_database import DoiDatabase
db = DoiDatabase(config,
                    config.get_int('crossref', 'scan_for_dois_after_year'),
                    config.get_int('crossref', 'scan_for_dois_before_year'))

In [8]:
from scan_database import ScanDatabase
scan_start_year = 2016
scan_end_year = 2016
reset_scan_database = False
rescore = False

scan_db = ScanDatabase(db, reset_scan_database=False)

In [9]:
scan_db.scan_pdfs(scan_start_year, scan_end_year, rescore=rescore)

: 

: 

In [8]:
query = """SELECT COUNT(*) FROM collections_papers.scans"""
conn.execute_query(query)

[(694282,)]

In [15]:
query = """SELECT COUNT(*) FROM thingy.scans"""
conn.execute_query(query)

CRITICAL:root:Bad SQL: 1049 (42000): Unknown database 'thingy':
SELECT COUNT(*) FROM thingy.scans


Traceback (most recent call last):
  File "/opt/anaconda3/envs/citations-finder/lib/python3.10/site-packages/mysql/connector/connection_cext.py", line 661, in cmd_query
    self._cmysql.query(
_mysql_connector.MySQLInterfaceError: Unknown database 'thingy'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/var/folders/s4/l2klswfj1wvcw_vbmvvbc70h0000gn/T/ipykernel_62268/20107826.py", line 108, in execute_query
    cursor.execute(query)
  File "/opt/anaconda3/envs/citations-finder/lib/python3.10/site-packages/mysql/connector/cursor_cext.py", line 374, in execute
    result = self._cnx.cmd_query(
  File "/opt/anaconda3/envs/citations-finder/lib/python3.10/site-packages/mysql/connector/opentelemetry/context_propagation.py", line 74, in wrapper
    return method(cnx, *args, **kwargs)
  File "/opt/anaconda3/envs/citations-finder/lib/python3.10/site-packages/mysql/connector/connection_cext.py", line 669, in cmd_query
    raise ge

ProgrammingError: 1049 (42000): Unknown database 'thingy'